# Set Up Environment

In [ ]:
# cell 1
# initialize all packages used for deployment of OMM SCOREs
# use 
# pip install -r _deployment/requirement.txt
# to install all dependencies

from iconservice import AddressPrefix, Address
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder,DepositTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass
from dotenv import dotenv_values

deployment_status = {}
print(1)

In [ ]:
# cell 2
config = dotenv_values("./_deployment/.env.jupyter")

EXA = 10 ** 18
ICX = 10 ** 18

NETWORK = config["NETWORK"]

PREPS_LIST_FILE = config["PREPS_LIST_FILE"]
with open(PREPS_LIST_FILE, "r") as file:
    PREP_LIST = json.load(file)

TIMESTAMP = int(config["TIMESTAMP"])

GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
WORKER_WALLET_ADDRESS=config["WORKER_WALLET_ADDRESS"]


#lending and borrow reward
LENDING_BORROW_PERCENTAGE = int(config["LENDING_BORROW_PERCENTAGE"]) * EXA // 100
#LP and OMM staking reward
LP_OMM_STAKING_PERCENTAGE = int(config["LP_OMM_STAKING_PERCENTAGE"]) * EXA // 100
#DAO reward percentage
DAO_DIST_PERCENTAGE = int(config["DAO_DIST_PERCENTAGE"]) * EXA // 100
#Worker reward 
WORKER_DIST_PERCENTAGE = int(config["WORKER_DIST_PERCENTAGE"]) * EXA // 100

assert (LENDING_BORROW_PERCENTAGE + LP_OMM_STAKING_PERCENTAGE + DAO_DIST_PERCENTAGE + WORKER_DIST_PERCENTAGE) // EXA == 1

ICX_EMISSION = int(config["ICX_PERCENTAGE"])
OICX_EMISSION = int(config["OICX_EMISSION"]) * ICX_EMISSION * EXA // 10000
DICX_EMISSION = int(config["DICX_EMISSION"]) * ICX_EMISSION * EXA // 10000

USDS_EMISSION = int(config["USDS_PERCENTAGE"])
OUSDS_EMISSION = int(config["OUSDS_EMISSION"]) * USDS_EMISSION * EXA // 10000
DUSDS_EMISSION = int(config["DUSDS_EMISSION"]) * USDS_EMISSION * EXA // 10000

IUSDC_EMISSION = int(config["IUSDC_PERCENTAGE"])
OIUSDC_EMISSION = int(config["OIUSDC_EMISSION"]) * IUSDC_EMISSION * EXA // 10000
DIUSDC_EMISSION = int(config["DIUSDC_EMISSION"]) * IUSDC_EMISSION * EXA // 10000

assert (OICX_EMISSION + DICX_EMISSION + OUSDS_EMISSION + DUSDS_EMISSION + OIUSDC_EMISSION + DIUSDC_EMISSION) // EXA == 1

#LP and OMM staking reward

OMM_SICX_DIST_PERCENTAGE = int(config["OMM_SICX_DIST_PERCENTAGE"]) * EXA // 100
OMM_USDS_DIST_PERCENTAGE = int(config["OMM_USDS_DIST_PERCENTAGE"]) * EXA // 100
OMM_USDC_DIST_PERCENTAGE = int(config["OMM_USDC_DIST_PERCENTAGE"]) * EXA // 100
OMM_DIST_PERCENTAGE = int(config["OMM_DIST_PERCENTAGE"]) * EXA // 100

assert (OMM_SICX_DIST_PERCENTAGE + OMM_USDS_DIST_PERCENTAGE + OMM_USDC_DIST_PERCENTAGE + OMM_DIST_PERCENTAGE) // EXA == 1

DEPOSIT_ICX_AMOUNT = int(config["DEPOSIT_ICX_AMOUNT"])
FEE_SHARING_TX_LIMIT = int(config["FEE_SHARING_TX_LIMIT"])
LOAN_ORIGINATION_PERCENTAGE = int(float(config["LOAN_ORIGINATION_PERCENTAGE"]) * EXA)
MINIMUM_OMM_STAKE = int(config["MINIMUM_OMM_STAKE"]) * EXA
OMM_UNSTAKING_PERIOD = int(config["OMM_UNSTAKING_PERIOD"])


DEPLOYER_WALLET_KEYSTORE = config["DEPLOYER_WALLET_KEYSTORE"]
POOL_CONFIGURATION_WALLET_KEYSTORE = config["POOL_CONFIGURATION_WALLET_KEYSTORE"]
CONTRACT_ADDRESS_FILE = config["CONTRACT_ADDRESS_FILE"]
AIRDROP_HOT_WALLET_ADDRESS=config["AIRDROP_HOT_WALLET_ADDRESS"]

BORROW_THRESHOLD=int(config["BORROW_THRESHOLD"])*EXA//100

_contract_file_loaded = False
print(1)

In [ ]:
# cell 3
connections = {
    "mainnet": {"iconservice": "https://ctz.solidwallet.io", "nid": 1},
    "yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
    "euljiro": {"iconservice": "https://test-ctz.solidwallet.io", "nid": 2},
    "pagoda": {"iconservice": "https://zicon.net.solidwallet.io", "nid": 80},
    "custom": {"iconservice": "http://18.237.205.52:9000/", "nid": 3},
    "local": {"iconservice": "http://localhost:9000/", "nid": 3},
    "sejong": {"iconservice": "https://sejong.net.solidwallet.io", "nid": 83}
}

env = connections[NETWORK]

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]


@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result


print(f"network -> {NETWORK}")
print(f"nid -> {NID}")

In [ ]:
# cell 4
# initialized all methods
def get_status_and_print(to, method, response) -> int:
    """
    get status and print response
    :param to:
    :param method:
    :param response:
    :return: status
    """
    status = response['status']
    print(f'SCORE {to} ::: Method {method} ::: Status: {status}')
    if len(response["eventLogs"]) > 0:
        for item in response["eventLogs"]:
            print(f'{item} \n')
    if status == 0:
        print(f'Failure: {response["failure"]}')
    return status


def _deploy_contract(_contract_name, params):
    deploy_transaction = DeployTransactionBuilder() \
        .from_(deployer_wallet.get_address()) \
        .to(GOVERNANCE_ADDRESS) \
        .nid(NID) \
        .nonce(100) \
        .content_type("application/zip") \
        .content(gen_deploy_data_content(_contract_name)) \
        .params(params) \
        .build()
    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)
    res = get_tx_result(tx_hash)
    status = get_status_and_print(_contract_name, "deploy", res)

    return {
        "address": res.get('scoreAddress', ''),
        "status": status
    }


def deploy_contract(_contract_name):
    """
    deploy all contracts excepts oToken and dToken
    :param _contract_name:  name of contract - should match with package name
    :return: SCORE address
    """
    params = {}
    if _contract_name == "workerToken":
        params = {'_initialSupply': int(config.get("worker_token_initial")), '_decimals': 18}
    if _contract_name == "rewardDistribution":
        DISTRIBUTION_PERCENTAGE = [
            {"recipient": "worker", "percentage": f'{WORKER_DIST_PERCENTAGE}'},
            {"recipient": "daoFund", "percentage": f'{DAO_DIST_PERCENTAGE}'},
            {"recipient": "lendingBorrow", "percentage": f'{LENDING_BORROW_PERCENTAGE}'},
            {"recipient": "liquidityProvider", "percentage": f'{LP_OMM_STAKING_PERCENTAGE}'}
        ]
        params = {"_distPercentage": DISTRIBUTION_PERCENTAGE, "_startTimestamp": TIMESTAMP}

    if _contract_name not in ['addressProvider', "workerToken"]:
        params['_addressProvider'] = contracts['addressProvider']

    result = _deploy_contract(_contract_name, params)
    deployment_status[f"deploy_contract_{_contract_name}"] = result["status"]
    return result["address"]

def deposit_transaction(_from, _to, _value, _method="add"):
    transaction = DepositTransactionBuilder() \
        .from_(_from.get_address()) \
        .to(contracts[_to]) \
        .nid(3) \
        .value(_value) \
        .step_limit(1000000000) \
        .nonce(100) \
        .action(_method) \
        .build()
    
    signed_transaction = SignedTransaction(transaction, _from)
    tx_hash = icon_service.send_transaction(signed_transaction)
    response = get_tx_result(tx_hash)
    status = get_status_and_print(_to, _method, response)
    return status

def deploy_token_contract(token, params):
    """
    deploy  oTokens and dTokens
    :param token: token name
    :param params: parameter
    :return: SCORE address
    """
    name = params["name"]
    symbol = params["symbol"]
    decimals = params["decimals"]
    print(f"deploying {name}::{symbol}::{decimals}")
    params = {"_name": name, "_symbol": symbol, "_decimals": decimals, '_addressProvider': contracts['addressProvider']}

    result = _deploy_contract(token, params)
    deployment_status[f"deploy_{token}_contract_{symbol}"] = result["status"]
    return result["address"]


def send_tx(_to, _method, _params, _value, _wallet) -> int:
    """
    send transaction to _to
    :param _to: destination conract name
    :param _method: method
    :param _params:  parameters
    :param _value:  value
    :param _wallet: transaction wallet
    :return: status of transaction
    """
    print(f'Calling {_method}, with parameters {_params} on the {_to} contract.')
    transaction = CallTransactionBuilder() \
        .from_(_wallet.get_address()) \
        .to(contracts[_to]) \
        .value(_value) \
        .step_limit(10000000) \
        .nid(NID) \
        .nonce(100) \
        .method(_method) \
        .params(_params) \
        .build()
    signed_transaction = SignedTransaction(transaction, _wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(f"{_to} ::> {_method} ::> {tx_hash}")
    response = get_tx_result(tx_hash)
    status = get_status_and_print(_to, _method, response)
    return status


### Import deployer wallet

In [ ]:
# cell 5
import getpass

keystore_password = getpass.getpass(f'Password for deployer wallet keystore : ')
deployer_wallet = KeyWallet.load(DEPLOYER_WALLET_KEYSTORE, keystore_password)
print(deployer_wallet.get_address())
print(icon_service.get_balance(deployer_wallet.get_address()))

In [ ]:
# cell 5.1 
# configuration wallet
import getpass

keystore_password = getpass.getpass(f'Password for configuration wallet keystore : ')
config_wallet = KeyWallet.load(POOL_CONFIGURATION_WALLET_KEYSTORE, keystore_password)
print(config_wallet.get_address())
print(icon_service.get_balance(config_wallet.get_address()))

## Initialized external SCORE addresses

In [ ]:
# cell 6
# Run this in case you want to import previous contract addresses
contracts_file_custom = config["CONTRACT_ADDRESS_FILE"]
with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)
    _contract_file_loaded = True

In [ ]:
# cell 7
if not _contract_file_loaded:
    contracts = {}
    contracts['sicx'] = config["SICX"]
    contracts['staking'] = config["STAKING"]
    contracts['iusdc'] = config["IUSDC"]
    contracts["usds"] = config.get("USDS")
    contracts['bandOracle'] = config["BAND_ORACLE"]
    contracts["dex"] = config["DEX"]

## Deploying all contracts

In [ ]:
# cell 8
#deploy addresss provider contract
if not _contract_file_loaded:
    contracts['addressProvider'] = deploy_contract('addressProvider')

In [ ]:
# cell 9
assert deployment_status.get('deploy_contract_addressProvider',0) == 1,"addressProvider is not deployed." 

In [ ]:
# cell 10
#deploy rest of contracts
if not _contract_file_loaded:
    deploy = ['daoFund', 'delegation', 'feeProvider', 'governance',
              'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider',
              'liquidationManager', 'ommToken', 'priceOracle', 'rewardDistribution',
              'workerToken', 'stakedLp']

    directory = os.getcwd()
    for item in deploy:
        print(f"\n Deploying {item} contract\n")
        contracts[item] = deploy_contract(item)

In [ ]:
# cell 11
assert deployment_status.get('deploy_contract_addressProvider',0) == 1,"addressProvider is not deployed."
assert deployment_status.get('deploy_contract_daoFund',0) == 1,"daoFund is not deployed."
assert deployment_status.get('deploy_contract_delegation',0) == 1,"delegation is not deployed."
assert deployment_status.get('deploy_contract_feeProvider',0) == 1,"feeProvider is not deployed."
assert deployment_status.get('deploy_contract_governance',0) == 1,"governance is not deployed."
assert deployment_status.get('deploy_contract_lendingPool',0) == 1,"lendingPool is not deployed."
assert deployment_status.get('deploy_contract_lendingPoolCore',0) == 1,"lendingPoolCore is not deployed."
assert deployment_status.get('deploy_contract_lendingPoolDataProvider',0) == 1,"lendingPoolDataProvider is not deployed."
assert deployment_status.get('deploy_contract_liquidationManager',0) == 1,"liquidationManager is not deployed."
assert deployment_status.get('deploy_contract_ommToken',0) == 1,"ommToken is not deployed."
assert deployment_status.get('deploy_contract_priceOracle',0) == 1,"priceOracle is not deployed."
assert deployment_status.get('deploy_contract_rewardDistribution',0) == 1,"rewardDistribution is not deployed."
assert deployment_status.get('deploy_contract_workerToken',0) == 1,"workerToken is not deployed."
assert deployment_status.get('deploy_contract_stakedLp',0) == 1,"stakedLp is not deployed."
print("-----Core contracts deployed-----")

## Retry contract deploy

In [ ]:
# cell 12
if not _contract_file_loaded:
    for item in deploy:
        if item not in contracts.keys() or contracts[item] == "" or contracts[item] == None:
            contracts[item] = deploy_contract(item)

## reading configuration for otoken and dtoken

In [ ]:
# cell 13
# deploy all oTokens and dTokens
if not _contract_file_loaded:
    TOKEN_CONFIGURATION_FILE = config["TOKEN_CONFIGURATION_FILE"]
    with open(TOKEN_CONFIGURATION_FILE, "r") as file:
        data = json.load(file)

    for params in data['dToken']:
        contracts[params['symbol']] = deploy_token_contract('dToken', params)

    for params in data['oToken']:
        contracts[params['symbol']] = deploy_token_contract('oToken', params)

In [ ]:
# cell 14
# check if all contracts are deployed or not
assert deployment_status.get('deploy_contract_addressProvider',0) == 1,"addressProvider is not deployed."
assert deployment_status.get('deploy_contract_daoFund',0) == 1,"daoFund is not deployed."
assert deployment_status.get('deploy_contract_delegation',0) == 1,"delegation is not deployed."
assert deployment_status.get('deploy_contract_feeProvider',0) == 1,"feeProvider is not deployed."
assert deployment_status.get('deploy_contract_governance',0) == 1,"governance is not deployed."
assert deployment_status.get('deploy_contract_lendingPool',0) == 1,"lendingPool is not deployed."
assert deployment_status.get('deploy_contract_lendingPoolCore',0) == 1,"lendingPoolCore is not deployed."
assert deployment_status.get('deploy_contract_lendingPoolDataProvider',0) == 1,"lendingPoolDataProvider is not deployed."
assert deployment_status.get('deploy_contract_liquidationManager',0) == 1,"liquidationManager is not deployed."
assert deployment_status.get('deploy_contract_ommToken',0) == 1,"ommToken is not deployed."
assert deployment_status.get('deploy_contract_priceOracle',0) == 1,"priceOracle is not deployed."
assert deployment_status.get('deploy_contract_rewardDistribution',0) == 1,"rewardDistribution is not deployed."
assert deployment_status.get('deploy_contract_workerToken',0) == 1,"workerToken is not deployed."
assert deployment_status.get('deploy_contract_stakedLp',0) == 1,"stakedLp is not deployed."
assert deployment_status.get('deploy_dToken_contract_dUSDS',0) == 1,"dUSDS is not deployed."
assert deployment_status.get('deploy_dToken_contract_dICX',0) == 1,"dICX is not deployed."
assert deployment_status.get('deploy_dToken_contract_dIUSDC',0) == 1,"dIUSDC is not deployed."
assert deployment_status.get('deploy_oToken_contract_oUSDS',0) == 1,"oUSDS is not deployed."
assert deployment_status.get('deploy_oToken_contract_oICX',0) == 1,"oICX is not deployed."
assert deployment_status.get('deploy_oToken_contract_oIUSDC',0) == 1,"oIUSDC is not deployed."
print("----All SCORE deployed successfully----")

In [ ]:
# cell 15
# create dummy contract address for liquidity pools
contracts['OMM/sICX'] = str(Address.from_data(AddressPrefix.CONTRACT,
                                          f"{contracts['ommToken']}/{contracts['sicx']}".encode("UTF-8")))
contracts['OMM/USDS'] = str(Address.from_data(AddressPrefix.CONTRACT,
                                          f"{contracts['ommToken']}/{contracts['usds']}".encode("UTF-8")))
contracts['OMM/IUSDC'] = str(Address.from_data(AddressPrefix.CONTRACT,
                                           f"{contracts['ommToken']}/{contracts['iusdc']}".encode("UTF-8")))

## Saving the pkl file


In [ ]:
# cell 16
# dump contract addresses to pkl file for future reference
now = datetime.utcnow()
filename = f'{NETWORK}_contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'

print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)

deployment_status["pkl"] = filename

## Deposit fee to Lending Pool

In [ ]:
# cell 17
# deposit fee to lending pool to be used for fee sharing
status = deposit_transaction(deployer_wallet, 'lendingPool', DEPOSIT_ICX_AMOUNT * EXA, "add")
deployment_status['deposit_fee_for_sharing'] = status

**Configure wallet for worker token**

In [ ]:
# cell 18
# set worker token receipient address using transfer method

params = {
    "_to": WORKER_WALLET_ADDRESS,
    "_value": 100*EXA
}
status= send_tx('workerToken', 'transfer', params, 0, deployer_wallet)
deployment_status['set wallet address to worker token'] = status

# Configuring SCOREs's parameters

In [ ]:
# cell 19
# initialzied all address through address provider contract
contract_details = [
    {'name': 'addressProvider', 'address': contracts['addressProvider']},
    {'name': 'daoFund', 'address': contracts['daoFund']},
    {'name': 'delegation', 'address': contracts['delegation']},
    {'name': 'feeProvider', 'address': contracts['feeProvider']},
    {'name': 'governance', 'address': contracts['governance']},
    {'name': 'lendingPool', 'address': contracts['lendingPool']},
    {'name': 'lendingPoolCore', 'address': contracts['lendingPoolCore']},
    {'name': 'lendingPoolDataProvider', 'address': contracts['lendingPoolDataProvider']},
    {'name': 'liquidationManager', 'address': contracts['liquidationManager']},
    {'name': 'ommToken', 'address': contracts['ommToken']},
    {'name': 'priceOracle', 'address': contracts['priceOracle']},
    {'name': 'bandOracle', 'address': contracts['bandOracle']},
    {'name': 'bridgeOToken', 'address': contracts['oUSDS']},
    {'name': 'rewards', 'address': contracts['rewardDistribution']},
    {'name': 'workerToken', 'address': contracts['workerToken']},
    {'name': 'sICX', 'address': contracts['sicx']},
    {'name': 'USDS', 'address': contracts['usds']},
    {'name': 'IUSDC', 'address': contracts['iusdc']},
    {'name': 'staking', 'address': contracts['staking']},
    {'name': 'oUSDS', 'address': contracts['oUSDS']},
    {'name': 'dUSDS', 'address': contracts['dUSDS']},
    {'name': 'oICX', 'address': contracts['oICX']},
    {'name': 'dICX', 'address': contracts['dICX']},
    {'name': 'oIUSDC', 'address': contracts['oIUSDC']},
    {'name': 'dIUSDC', 'address': contracts['dIUSDC']},
    {'name': 'stakedLP', 'address': contracts['stakedLp']},
    {'name': 'dex', 'address': contracts['dex']}
]

setting_addressProvider = [
    {'contract': 'addressProvider', 'method': 'setAddresses', 'params': {'_addressDetails': contract_details}},
    {'contract': 'addressProvider', 'method': 'setSCOREAddresses', 'params': {}},
]

for sett in setting_addressProvider:
    print(f'\n')
    status= send_tx(sett['contract'], sett['method'], sett['params'], 0, deployer_wallet)
    deployment_status[f"{sett['method']} in {sett['contract']}"] = status


In [ ]:
# cell 20
# verify if all addresses on each SCORE are configured
assert deployment_status['setAddresses in addressProvider'] == 1
assert deployment_status['setSCOREAddresses in addressProvider'] == 1
print("ADDRESS PROVIDER CONFIGURED CORRECTLY")

In [ ]:
# cell 21
# configure contrats
settings = [
    {'key': 'setFeeSharingTxnLimit', 'contract': 'lendingPool', 'method': 'setFeeSharingTxnLimit',
     'params': {'_limit': FEE_SHARING_TX_LIMIT}},
    {'key': 'setLoanOriginationFeePercentage', 'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage',
     'params': {'_percentage': LOAN_ORIGINATION_PERCENTAGE}},
    {'key': 'setSymbol (usds)', 'contract': 'lendingPoolDataProvider', 'method': 'setSymbol',
     'params': {'_reserve': contracts['usds'], '_sym': "USDS"}},
    {'key': 'setSymbol (sicx)', 'contract': 'lendingPoolDataProvider', 'method': 'setSymbol',
     'params': {'_reserve': contracts['sicx'], '_sym': "ICX"}},
    {'key': 'setSymbol (iusdc)', 'contract': 'lendingPoolDataProvider', 'method': 'setSymbol',
     'params': {'_reserve': contracts['iusdc'], '_sym': "USDC"}},
    {'key': 'addAllContributors', 'contract': 'delegation', 'method': 'addAllContributors',
     'params': {'_preps': PREP_LIST}},
    {'key': 'setMinimumStake', 'contract': 'ommToken', 'method': 'setMinimumStake',
     'params': {'_min': f'{MINIMUM_OMM_STAKE}'}},
    {'key': 'setUnstakingPeriod', 'contract': 'ommToken', 'method': 'setUnstakingPeriod',
     'params': {'_timeInSeconds': f'{OMM_UNSTAKING_PERIOD}'}}
]

for sett in settings:
    print(f'\n')
    status= send_tx(sett['contract'], sett['method'], sett['params'], 0, deployer_wallet)
    deployment_status[f"{sett['key']} in {sett['contract']}"] = status

In [ ]:
# cell 22
# configuring reserves constants
settings_reserves = [
    {'contract': 'governance',
     'key': 'setReserveConstants (usds)',
     'method': 'setReserveConstants',
     'params': {"_constants": [{"reserve": contracts['usds'],
                                "optimalUtilizationRate": f"8{'0' * 17}",
                                "baseBorrowRate": f"2{'0' * 16}",
                                "slopeRate1": f"6{'0' * 16}",
                                "slopeRate2": f"2{'0' * 18}"}]}},
    {'contract': 'governance',
     'key': 'setReserveConstants (sicx)',
     'method': 'setReserveConstants',
     'params': {"_constants": [{"reserve": contracts['sicx'],
                                "optimalUtilizationRate": f"8{'0' * 17}",
                                "baseBorrowRate": f"0{'0' * 17}",
                                "slopeRate1": f"8{'0' * 16}",
                                "slopeRate2": f"4{'0' * 18}"}]}},
    {'contract': 'governance',
     'key': 'setReserveConstants (iusdc)',
     'method': 'setReserveConstants',
     'params': {"_constants": [{"reserve": contracts['iusdc'],
                                "optimalUtilizationRate": f"8{'0' * 17}",
                                "baseBorrowRate": f"2{'0' * 16}",
                                "slopeRate1": f"6{'0' * 16}",
                                "slopeRate2": f"2{'0' * 18}"}]}}
]

for sett in settings_reserves:
    print(f'\n')
    status= send_tx(sett['contract'], sett['method'], sett['params'], 0, deployer_wallet)
    deployment_status[f"{sett['key']} in {sett['contract']}"] = status

## Adding USDS reserve in LendingPoolCore via Governance



In [ ]:
# cell 23
# initialize usds reserve
settings = [
    {
        "method": "initializeReserve",
        "params": {
            "_reserve": {
                "reserveAddress": contracts['usds'],
                "oTokenAddress": contracts['oUSDS'],
                "dTokenAddress": contracts['dUSDS'],
                "lastUpdateTimestamp": "0",
                "liquidityRate": "0",
                "borrowRate": "0",
                "liquidityCumulativeIndex": f"1{'0' * 18}",
                "borrowCumulativeIndex": f"1{'0' * 18}",
                "baseLTVasCollateral": "500000000000000000",
                "liquidationThreshold": "650000000000000000",
                "liquidationBonus": "100000000000000000",
                "decimals": "18",
                "borrowingEnabled": "1",
                "usageAsCollateralEnabled": "1",
                "isFreezed": "0",
                "isActive": "1"
            }
        }
    },
    {
        "method": "updateBorrowThreshold",
        "params": {
            "_reserve": contracts['usds'],
            "_borrowThreshold": BORROW_THRESHOLD

        }
    }
]

for setting in settings:
    print(f'\n')
    status = send_tx('governance', setting["method"], setting["params"], 0, deployer_wallet)
    deployment_status[f"{setting['method']} in governance (usds)"] = status

## Adding ICX reserve

In [ ]:
# cell 24
# initialize sicx reserve
settings = [
    {
        "method": "initializeReserve",
        "params": {
            "_reserve": {
                "reserveAddress": contracts['sicx'],
                "oTokenAddress": contracts['oICX'],
                "dTokenAddress": contracts['dICX'],
                "lastUpdateTimestamp": "0",
                "liquidityRate": "0",
                "borrowRate": "0",
                "liquidityCumulativeIndex": f"1{'0' * 18}",
                "borrowCumulativeIndex": f"1{'0' * 18}",
                "baseLTVasCollateral": "500000000000000000",
                "liquidationThreshold": "650000000000000000",
                "liquidationBonus": "100000000000000000",
                "decimals": "18",
                "borrowingEnabled": "1",
                "usageAsCollateralEnabled": "1",
                "isFreezed": "0",
                "isActive": "1"
            }
        }
    },
    {
        "method": "updateBorrowThreshold",
        "params": {
            "_reserve": contracts['sicx'],
            "_borrowThreshold": BORROW_THRESHOLD

        }
    }
]

for setting in settings:
    print(f'\n')
    status = send_tx('governance', setting["method"], setting["params"], 0, deployer_wallet)
    deployment_status[f"{setting['method']} in governance (sicx)"] = status

## Adding IUSDC reserve

In [ ]:
# cell 25
# initialize iusdc reserve
settings = [
    {
        "method": "initializeReserve",
        "params": {
            "_reserve": {
                "reserveAddress": contracts['iusdc'],
                "oTokenAddress": contracts['oIUSDC'],
                "dTokenAddress": contracts['dIUSDC'],
                "lastUpdateTimestamp": "0",
                "liquidityRate": "0",
                "borrowRate": "0",
                "liquidityCumulativeIndex": f"1{'0' * 18}",
                "borrowCumulativeIndex": f"1{'0' * 18}",
                "baseLTVasCollateral": "500000000000000000",
                "liquidationThreshold": "650000000000000000",
                "liquidationBonus": "100000000000000000",
                "decimals": "6",
                "borrowingEnabled": "1",
                "usageAsCollateralEnabled": "1",
                "isFreezed": "0",
                "isActive": "1"
            }
        }
    },
    {
        "method": "updateBorrowThreshold",
        "params": {
            "_reserve": contracts['iusdc'],
            "_borrowThreshold": BORROW_THRESHOLD

        }
    }
]

for setting in settings:
    print(f'\n')
    status = send_tx('governance', setting["method"], setting["params"], 0, deployer_wallet)
    deployment_status[f"{setting['method']} in governance (iusdc)"] = status

In [ ]:
# cell 26
# configure rewards for lending borrow
params = {
    "_assetConfigs":
        [
            # reserves
            {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["oUSDS"],
             "assetName": "oUSDS",
             "distPercentage": f"{OUSDS_EMISSION}"},
            {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["dUSDS"],
             "assetName": "dUSDS",
             "distPercentage": f"{DUSDS_EMISSION}"},
            {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["dICX"],
             "assetName": "dICX",
             "distPercentage": f"{DICX_EMISSION}"},
            {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["oICX"],
             "assetName": "oICX",
             "distPercentage": f"{OICX_EMISSION}"},
            {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["oIUSDC"],
             "assetName": "oIUSDC", "distPercentage": f"{OIUSDC_EMISSION}"},
            {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["dIUSDC"],
             "assetName": "dIUSDC", "distPercentage": f"{DIUSDC_EMISSION}"},
        ]
}

status = send_tx('governance', 'addPools', params, 0, deployer_wallet)
deployment_status[f"addPools in governance"] = status

In [ ]:
# cell 27
# assert if all configurations are success or not
assert deployment_status['deploy_contract_addressProvider'] == 1
assert deployment_status['deploy_contract_daoFund'] == 1
assert deployment_status['deploy_contract_delegation'] == 1
assert deployment_status['deploy_contract_feeProvider'] == 1
assert deployment_status['deploy_contract_governance'] == 1
assert deployment_status['deploy_contract_lendingPool'] == 1
assert deployment_status['deploy_contract_lendingPoolCore'] == 1
assert deployment_status['deploy_contract_lendingPoolDataProvider'] == 1
assert deployment_status['deploy_contract_liquidationManager'] == 1
assert deployment_status['deploy_contract_ommToken'] == 1
assert deployment_status['deploy_contract_priceOracle'] == 1
assert deployment_status['deploy_contract_rewardDistribution'] == 1
assert deployment_status['deploy_contract_workerToken'] == 1
assert deployment_status['deploy_contract_stakedLp'] == 1
assert deployment_status['deploy_dToken_contract_dUSDS'] == 1
assert deployment_status['deploy_dToken_contract_dICX'] == 1
assert deployment_status['deploy_dToken_contract_dIUSDC'] == 1
assert deployment_status['deploy_oToken_contract_oUSDS'] == 1
assert deployment_status['deploy_oToken_contract_oICX'] == 1
assert deployment_status['deploy_oToken_contract_oIUSDC'] == 1
assert deployment_status['deposit_fee_for_sharing'] == 1
assert deployment_status['setFeeSharingTxnLimit in lendingPool'] == 1
assert deployment_status['setLoanOriginationFeePercentage in feeProvider'] == 1
assert deployment_status['setSymbol (usds) in lendingPoolDataProvider'] == 1
assert deployment_status['setSymbol (sicx) in lendingPoolDataProvider'] == 1
assert deployment_status['setSymbol (iusdc) in lendingPoolDataProvider'] == 1
assert deployment_status['addAllContributors in delegation'] == 1
assert deployment_status['setMinimumStake in ommToken'] == 1
assert deployment_status['setUnstakingPeriod in ommToken'] == 1
assert deployment_status['addPools in governance'] == 1
assert deployment_status['setReserveConstants (usds) in governance'] == 1
assert deployment_status['setReserveConstants (sicx) in governance'] == 1
assert deployment_status['setReserveConstants (iusdc) in governance'] == 1
assert deployment_status['initializeReserve in governance (usds)'] == 1
assert deployment_status['initializeReserve in governance (sicx)'] == 1
assert deployment_status['initializeReserve in governance (iusdc)'] == 1
assert deployment_status['updateBorrowThreshold in governance (usds)'] == 1
assert deployment_status['updateBorrowThreshold in governance (sicx)'] == 1
assert deployment_status['updateBorrowThreshold in governance (iusdc)'] == 1
assert deployment_status['set wallet address to worker token'] == 1
print("----Configured successfully----")
contracts

## Test deposit (ICX, USDS, iUSDC)

In [ ]:
# test 1
# test deposit sicx
params = {"_amount": 5 * 10 ** 18}
send_tx('lendingPool','deposit',params, params['_amount'], deployer_wallet)

In [ ]:
#test 2
# deposit USDS
_depositAmount = 5 * 10 ** 18
depositData = {'method': 'deposit', 'params': {'amount': _depositAmount}}
data = json.dumps(depositData).encode('utf-8')

params = {"_to": contracts['lendingPool'],
        "_value": _depositAmount, 
        "_data": data}

send_tx('usds','transfer', params, 0, deployer_wallet)

In [ ]:
# test 3
# deposit iUSDC
_depositAmount = 5 * 10 ** 6
depositData = {'method': 'deposit', 'params': {'amount': _depositAmount}}

data = json.dumps(depositData).encode('utf-8')
params = {"_to": contracts['lendingPool'],
        "_value": _depositAmount, 
        "_data": data}

send_tx('iusdc','transfer', params, 0, deployer_wallet)

## Test Borrow (ICX, USDS, iUSDC)

In [ ]:
# test 4
# borrow sicx
params = {"_reserve": contracts['sicx'], "_amount": 1 * 10 ** 18}
send_tx('lendingPool','borrow', params, 0, deployer_wallet)

In [ ]:
# test 5
# borrow USDS
params = {"_reserve": contracts['usds'],"_amount": 10**18}
send_tx('lendingPool','borrow', params, 0, deployer_wallet)

In [ ]:
# test 6
# borrow iUSDC
params = {"_reserve": contracts['iusdc'],"_amount": 10**6}
send_tx('lendingPool','borrow', params, 0, deployer_wallet)

## Start reward distribution

In [ ]:
contracts

In [ ]:
# cell 28
# start reward dsitriution on t0 day 

status= send_tx('rewardDistribution', 'startDistribution', {}, 0, deployer_wallet)


## Enable reward claim

In [ ]:
# cell 29
# enable reward claim  on start of t5

status= send_tx('governance', 'enableRewardClaim', {}, 0, deployer_wallet)

## Reward configurations

In [ ]:
# cell 30
# update distritbution percentage can only be called with 15 mins of day start
DISTRIBUTION_PERCENTAGE = [
    {"recipient": "worker", "percentage": f'{WORKER_DIST_PERCENTAGE}'},
    {"recipient": "daoFund", "percentage": f'{DAO_DIST_PERCENTAGE}'},
    {"recipient": "lendingBorrow", "percentage": f'{LENDING_BORROW_PERCENTAGE}'},
    {"recipient": "liquidityProvider", "percentage": f'{LP_OMM_STAKING_PERCENTAGE}'}
]
assert (WORKER_DIST_PERCENTAGE+DAO_DIST_PERCENTAGE+LENDING_BORROW_PERCENTAGE+LP_OMM_STAKING_PERCENTAGE)==EXA, "percentage distribution mismatched"
params = {"_distPercentage": DISTRIBUTION_PERCENTAGE}
status= send_tx('rewardDistribution', 'setDistributionPercentage', params, 0, deployer_wallet)


In [ ]:
# cell 31
# remove pool from reward distribution
asset = ''  # address of pool 

params = {
    "_asset": asset
}

status = send_tx('governance', removePool, params, 0, deployer_wallet)


In [ ]:
contracts

In [ ]:
# cell 32
# add or update existing pool
# please refer to cell 14 to generate dummy addresses for dex pools
# OMM_SICX_DIST_PERCENTAGE = int(config["OMM_SICX_DIST_PERCENTAGE"]) * EXA // 100  OMM/sICX
# OMM_USDS_DIST_PERCENTAGE = int(config["OMM_USDS_DIST_PERCENTAGE"]) * EXA // 100  OMM/USDS
# OMM_USDC_DIST_PERCENTAGE = int(config["OMM_USDC_DIST_PERCENTAGE"]) * EXA // 100  OMM/IUSDC
# OMM_DIST_PERCENTAGE = int(config["OMM_DIST_PERCENTAGE"]) * EXA // 100

poolID = -1 # pool id from dex set -1 for all other pools
rewardEntity = 'liquidityProvider'  # must be either  lendingBorrow or liquidityProvider
asset = contracts['ommToken']  # address of pool 
assetName = 'OMM'  # asset name
distPercentage = OMM_DIST_PERCENTAGE  # distribution percentage for pool of lendingBorrow or liquidityProvider
params = {
    "_assetConfig":
        {
            "poolID": f'{poolID}',
            "rewardEntity": rewardEntity,
            "asset": asset,
            "assetName": assetName,
            "distPercentage": f"{distPercentage}"
        }
}

status = send_tx('governance', "addPool", params, 0, deployer_wallet)


## Updating Contracts

In [ ]:
# Updating mainnet contracts
contract_ = 'priceOracle'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken'
elif contract_ == 'oUSDS':
    update = contracts['oUSDS']
    contract_ = 'oToken'
elif contract_ == 'oIUSDC':
    update = contracts['oIUSDC']
    contract_ = 'oToken'
elif contract_ == 'dICX':
    update = contracts['dICX']
    contract_ = 'dToken'
elif contract_ == 'dUSDS':
    update = contracts['dUSDS']
    contract_ = 'dToken'
elif contract_ == 'dIUSDC':
    update = contracts['dIUSDC']
    contract_ = 'dToken'
else:
    update = contracts[contract_]
# update = contracts[contract_]
print('update', contract_, update)


deploy_transaction = DeployTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(update) \
    .nid(NID) \
    .nonce(100) \
    .content_type("application/zip") \
    .content(gen_deploy_data_content(contract_)) \
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

response=get_tx_result(tx_hash)
get_status_and_print("contract_", "update_contract", response)

### Create pools

In [ ]:
# check required balance 

def _check_balance(_contract,_wallet):
    call = CallBuilder().from_(_wallet.get_address())\
                    .to(contracts[_contract])\
                    .method('balanceOf')\
                    .params({'_owner': _wallet.get_address()}) \
                    .build()
    result = icon_service.call(call)
    return int(result, 0)

assert _check_balance('sicx',config_wallet)//EXA>=10,"Insufficent sicx balance"
assert _check_balance('usds',config_wallet)//EXA>=1,"Insufficent usds balance"
assert _check_balance('iusdc',config_wallet)//10**6>=1,"Insufficent iusdc balance"
assert _check_balance('ommToken',config_wallet)//EXA>=15,"Insufficent ommToken balance"
print("balance check completed")


In [ ]:
# deposit sicx
data = "{\"method\": \"_deposit\"}".encode("utf-8")
_params = {'_to': contracts['dex'], '_value': 10 * EXA, '_data': data}
status = send_tx('sicx', 'transfer', _params, 0, config_wallet)


In [ ]:
# deposit omm token for sicx pair
data = "{\"method\": \"_deposit\"}".encode("utf-8")
_params = {'_to': contracts['dex'], '_value': 15 * EXA, '_data': data}
status = send_tx('ommToken', 'transfer', _params, 0, config_wallet)

In [ ]:
# create omm sicx pair in DEX
_params = {'_baseToken': contracts['ommToken'],
           "_quoteToken":contracts["sicx"],
           "_baseValue": 1276 * EXA//100 ,
           '_quoteValue': 10 * EXA}

status = send_tx('dex', 'add', _params, 0, config_wallet)


In [ ]:
#set pool id of OMM/sICX
pool_ids["OMM/sICX"]=0x19

In [ ]:
# deposit usds
data = "{\"method\": \"_deposit\"}".encode("utf-8")
_params = {'_to': contracts['dex'], '_value': 1 * EXA, '_data': data}
status = send_tx('usds', 'transfer', _params, 0, config_wallet)

In [ ]:
# deposit omm token for usds
data = "{\"method\": \"_deposit\"}".encode("utf-8")
_params = {'_to': contracts['dex'], '_value': 1 * EXA, '_data': data}
status = send_tx('ommToken', 'transfer', _params, 0, config_wallet)

In [ ]:
# create omm usds pair in DEX
_params = {'_baseToken': contracts['ommToken'],
           "_quoteToken":contracts["usds"],
           "_baseValue": 1 * EXA ,
           '_quoteValue': 1 * EXA}

status = send_tx('dex', 'add', _params, 0, config_wallet)


In [ ]:
#set id for OMM/USDS
pool_ids={}
pool_ids["OMM/USDS"]=0x17

In [ ]:
# deposit iusdc
data = "{\"method\": \"_deposit\"}".encode("utf-8")
_params = {'_to': contracts['dex'], '_value': 1 * 10**6, '_data': data}
status = send_tx('iusdc', 'transfer', _params, 0, config_wallet)

In [ ]:
# deposit omm token iusdc
data = "{\"method\": \"_deposit\"}".encode("utf-8")
_params = {'_to': contracts['dex'], '_value': 1 * EXA, '_data': data}
status = send_tx('ommToken', 'transfer', _params, 0, config_wallet)

In [ ]:
# create omm iusdc pair in DEX
_params = {'_baseToken': contracts['ommToken'],
           "_quoteToken":contracts["iusdc"],
           "_baseValue":1*EXA ,
           '_quoteValue': 1 * 10**6}

status = send_tx('dex', 'add', _params, 0, config_wallet)

In [ ]:
#set id for OMM/IUSDC
pool_ids["OMM/IUSDC"]=0x18

In [ ]:
#add OMM/sICX pool in OMM
poolID =pool_ids["OMM/sICX"]   # pool id from dex set -1 for all other pools
rewardEntity = 'liquidityProvider'  # must be either  lendingBorrow or liquidityProvider
asset = contracts['OMM/sICX']  # address of pool 
assetName = 'OMM/sICX'  # asset name
distPercentage = OMM_SICX_DIST_PERCENTAGE  # distribution percentage for pool of lendingBorrow or liquidityProvider
params = {
    "_assetConfig":
        {
            "poolID": f'{poolID}',
            "rewardEntity": rewardEntity,
            "asset": asset,
            "assetName": assetName,
            "distPercentage": f"{distPercentage}"
        }
}

status = send_tx('governance', "addPool", params, 0, deployer_wallet)

In [ ]:
pool_ids

In [ ]:
#add OMM/USDS pool in OMM
poolID = pool_ids["OMM/USDS"]   # pool id from dex set -1 for all other pools
rewardEntity = 'liquidityProvider'  # must be either  lendingBorrow or liquidityProvider
asset = contracts['OMM/USDS']  # address of pool 
assetName = 'OMM/USDS'  # asset name
distPercentage = OMM_USDS_DIST_PERCENTAGE  # distribution percentage for pool of lendingBorrow or liquidityProvider
params = {
    "_assetConfig":
        {
            "poolID": f'{poolID}',
            "rewardEntity": rewardEntity,
            "asset": asset,
            "assetName": assetName,
            "distPercentage": f"{distPercentage}"
        }
}

status = send_tx('governance', "addPool", params, 0, deployer_wallet)

In [ ]:
#add OMM/IUSDC pool in OMM
poolID = pool_ids["OMM/IUSDC"]  # pool id from dex set -1 for all other pools
rewardEntity = 'liquidityProvider'  # must be either  lendingBorrow or liquidityProvider
asset = contracts['OMM/IUSDC']  # address of pool 
assetName = 'OMM/IUSDC'  # asset name
distPercentage = OMM_USDC_DIST_PERCENTAGE  # distribution percentage for pool of lendingBorrow or liquidityProvider
params = {
    "_assetConfig":
        {
            "poolID": f'{poolID}',
            "rewardEntity": rewardEntity,
            "asset": asset,
            "assetName": assetName,
            "distPercentage": f"{distPercentage}"
        }
}

status = send_tx('governance', "addPool", params, 0, deployer_wallet)

In [ ]:
#transfer fund for Airdrop
_required_fund=952000*EXA
_hot_wallet_address=config['AIRDROP_HOT_WALLET_ADDRESS']
_params={
    "_value":f'{_required_fund}',
    "_address":_hot_wallet_address
}
send_tx('governance', "transferOmmFromDaoFund", _params, 0, deployer_wallet)

In [ ]:
#stake LP
# data = {
#     'method': "stake"
# }

# data = json.dumps(data).encode('utf-8')

# _params = {
#     '_to': contracts['stakedLp'],
#     '_value': 1*EXA,
#     '_id': f'{pool_ids["OMM/sICX"]}',
#     '_data': data
# }
# print("staking OMM/sICX")
# status = send_tx('dex', "transfer", _params, 0, config_wallet)

# _params = {
#     '_to': contracts['stakedLp'],
#     '_value': 9*EXA//10,
#     '_id': f'{pool_ids["OMM/USDS"]}',
#     '_data': data
# }
# print("staking OMM/USDS")
# status = send_tx('dex', "transfer", _params, 0, config_wallet)


# _params = {
#     '_to': contracts['stakedLp'],
#     '_value': 9*10**12//10,
#     '_id': f'{pool_ids["OMM/IUSDC"]}',
#     '_data': data
# }
# print("staking OMM/IUSDC")
# status = send_tx('dex', "transfer", _params, 0, config_wallet)
